In [1]:
import os
import cv2
import matplotlib.pyplot as plt 
import tensorflow as tf
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

In [2]:
image_size = 200

path1 = r'C:\Users\khara\Downloads\Mask_NoMask'
cate = ['Mask', 'No_Mask']

input_image = []
for i in cate:
    folders = os.path.join(path1, i)
    label = cate.index(i)                  # need to tell software which image is of which category
    for image in os.listdir(folders):
        image_path = os.path.join(folders, image)
        image_array = cv2.imread(image_path)    # using cv2, reading image storing variable
        image_array = cv2.resize(image_array, (image_size, image_size))    # resizing image to 200 * 200
        input_image.append([image_array, label])

In [3]:
len(input_image)

8838

In [4]:
input_image = input_image[2000:6000]

In [5]:
np.random.shuffle(input_image)

X  = []
Y =  []

for X_values , labels in input_image:
    X.append(X_values)
    Y.append(labels)

X = np.array(X)
Y = np.array(Y)

In [6]:
X_train = X[:3500]
X_test = X[3500:]

Y_train = Y[:3500]
Y_test = Y[3500:]

X_train = X_train/255
X_test = X_test/255
X_train.shape

(3500, 200, 200, 3)

In [7]:
X_test.shape

(500, 200, 200, 3)

In [8]:
Y_train.shape

(3500,)

In [9]:
model = Sequential()
model.add(Conv2D(filters = 16 , kernel_size =(5,5) , activation ='relu'  ,   ))
model.add(MaxPool2D(pool_size =(2,2)))
#model.add(Conv2D(filters = 16 , kernel_size =(5,5) , activation ='relu'  ,   ))
#model.add(MaxPool2D(pool_size =(2,2)))
model.add(Flatten())
#model.add(Dense(128 , activation = 'relu' ))
#model.add(Dropout(0.4))
model.add(Dense(2 , activation= 'softmax'))

In [10]:
model.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [11]:
model.fit(X_train, Y_train, epochs=15 ,validation_split= 0.15)

Epoch 1/15
93/93 [==============================] - 65s 670ms/step - loss: 1.1034 - accuracy: 0.7395 - val_loss: 0.3071 - val_accuracy: 0.8914
Epoch 2/15
93/93 [==============================] - 56s 606ms/step - loss: 0.2627 - accuracy: 0.9052 - val_loss: 0.2671 - val_accuracy: 0.9124
Epoch 3/15
93/93 [==============================] - 55s 593ms/step - loss: 0.1953 - accuracy: 0.9284 - val_loss: 0.2295 - val_accuracy: 0.9257
Epoch 4/15
93/93 [==============================] - 58s 621ms/step - loss: 0.1302 - accuracy: 0.9529 - val_loss: 0.2944 - val_accuracy: 0.8933
Epoch 5/15
93/93 [==============================] - 57s 614ms/step - loss: 0.0934 - accuracy: 0.9677 - val_loss: 0.2689 - val_accuracy: 0.8952
Epoch 6/15
93/93 [==============================] - 59s 630ms/step - loss: 0.0693 - accuracy: 0.9795 - val_loss: 0.2454 - val_accuracy: 0.9086
Epoch 7/15
93/93 [==============================] - 60s 640ms/step - loss: 0.0529 - accuracy: 0.9859 - val_loss: 0.2680 - val_accuracy: 0.9162

In [12]:
pred = model.predict(X_test)
pred_classes = np.argmax(pred, 1)

In [13]:
from sklearn.metrics import confusion_matrix

In [14]:
tab = confusion_matrix(pred_classes, Y_test)
tab

array([[359,  22],
       [  9, 110]], dtype=int64)

In [15]:
acc = tab.diagonal().sum()*100/tab.sum()
acc

93.8

In [16]:
model.save("Mask_Nomask.h5")        ## to save the model

In [17]:
from keras.models import load_model   ## required import

In [18]:
abcd = load_model("Mask_Nomask.h5")

In [19]:
import cv2
import numpy as np
from keras.models import load_model
#model = load_model("F:\pranjal\knowledge\data analytics\R programing\conducting classes\PythonModel\my_mask_NoMask_model.h5") 
model=load_model("Mask_Nomask.h5")

results={0:'Mask  found',1:'Mask not found '}
GR_dict={0:(0,255,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 

haarcascade = cv2.CascadeClassifier(r"D:\harcascing\haarcascade_frontalface_alt.xml")
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1)

    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(200,200))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,200,200,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2)

    cv2.imshow('Liv Camera',   im)
    key = cv2.waitKey(10)
    
    if key == 27: # use the escape key
        break

cap.release()

cv2.destroyAllWindows()
